<a href="https://colab.research.google.com/github/ayan-de/Recipe-RAG/blob/main/RAG_with_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# Import the userdata module from Google Colab
from google.colab import userdata
# Retrieve the API key stored under 'openAi_key' from Colab's userdata
api_key = userdata.get('openAi_key')

In [ ]:
# Mount the drive - Connect to local drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change directory to this folder
%cd /content/drive/MyDrive/RAG-ztm/GenAI/RAG/RAG with OpenAI

/content/drive/MyDrive/RAG-ztm/GenAI/RAG/RAG with OpenAI


# Perform OCR and transform to images

In [ ]:
# Install the pdf2image library for converting PDF files to images
!pip install pdf2image
# Install the poppler-utils package, required by pdf2image to work with PDF files
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.7 [186 kB]
Fetched 186 kB in 1s (356 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.7_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.7) ...
Setting up poppler-utils (22.02.0-2ubuntu0.7) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
# Import the libraries
from pdf2image import convert_from_path
import os

In [ ]:
# Function to converts pdfs into images and stores the image paths
def pdf_to_images(pdf_path, output_folder):
  # Create the output folder if it doesn't exist
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)

  # Convert PDF into images
  images = convert_from_path(pdf_path) # Convert each page of the PDF to an image
  image_paths = []

  # Save images and store their paths
  for i, image in enumerate(images):
    image_path = os.path.join(output_folder, f"page{i+1}.jpg") # Generate the image file path
    image.save(image_path, "JPEG") # Save the image as a JPEG file
    image_paths.append(image_path) # Append the image path to the list

  return image_paths # Return the list of image paths

In [ ]:
# Define the path to the PDF and the output folder for images
pdf_path = "Things mother used to make.pdf"
output_folder = "images"

# Convert the PDF into images and store the image paths
image_paths = pdf_to_images(pdf_path, output_folder)

# **Now we will use GPT to extract info form images from the above section**

In [ ]:
# Install the openAI library
!pip install openai

In [ ]:
# Import the libraries
from openai import OpenAI
import base64

In [ ]:
# Set up connection to OpenAI API
client = OpenAI(
    api_key=api_key, # Use the provided API key for authentication
base_url="https://models.inference.ai.azure.com" # Set the base URL for the API
)
# Specify the model to be used
# model = "openai/gpt-4.1"
model = "gpt-4o-mini"

In [ ]:
# Read and encode one image
image_path = "images/page23.jpg" # Path to the image to be encoded

# Encode the image in base64 and decode to string
with open(image_path, "rb") as image_file:
  image_data = base64.b64encode(image_file.read()).decode('utf-8')
image_data

'...'

In [ ]:
# Define the system prompt
system_prompt = """
Please analyze the content of this image and extract any related recipe information.
"""

In [ ]:
# Call the OpenAI API use the chat completion method
response = client.chat.completions.create(
    model = model,
    messages = [
        # Provide the system prompt
        {"role": "system", "content": system_prompt},

        # The user message contains both the text and image URL / path
        {"role": "user", "content": [
            {"type": "text", "text": "This is the image from the recipe page."},
            {"type": "image_url",
             "image_url": {"url": f"data:image/jpeg;base64,{image_data}",
                           "detail": "low"}}
        ]}
    ]
)

In [ ]:
# Retrieve the content
gpt_response = response.choices[0].message.content

In [ ]:
from IPython.display import Markdown, display

# Display the GPT response as Markdown
display(Markdown(gpt_response))

Based on the content provided in the image, here are the extracted recipe details for the two types of bread included:

### Bannocks
**Ingredients:**
- 1 Cupful of Thick Sour Milk
- 1/2 Cupful of Sugar
- 1 Egg
- 2 Cupfuls of Flour
- 1/2 Cupful of Indian Meal
- 1 Teaspoonful of Baking Powder
- A pinch of Salt

**Instructions:**
1. Mix the ingredients until the mixture is stiff enough to drop from a spoon.
2. Drop the mixture, size of a walnut, into boiling fat.
3. Serve warm, with maple syrup.

---

### Boston Brown Bread
**Ingredients:**
- 1 Cupful of Rye Meal
- 1 Cupful of Graham Meal
- 1 Cupful of Indian Meal
- 1 Cupful of Flour
- 1 Cupful of Sour Milk
- 1 Cupful of Sweet Milk
- 1 Cupful of Molasses
- 1 Teaspoonful of Baking Soda

**Instructions:**
1. Mix all the ingredients together.
2. Beat the soda into the molasses until it foams; add the mixture to the rest.
3. Transfer into a tin pail which has been well greased.
4. If using a steamer, do not cover it tightly and steam for about 3 hours, but if not, bake in an oven until done.

Feel free to ask if you need more information or clarifications about these recipes!

In [ ]:
# Define a function to get the GPT response and display it in Markdown
def get_gpt_response():
  gpt_response = response.choices[0].message.content # Extract the response content from the API response
  return display(Markdown(gpt_response)) # Display the response as Markdown

# Call the function to display the GPT response
get_gpt_response()

Based on the content provided in the image, here are the extracted recipe details for the two types of bread included:

### Bannocks
**Ingredients:**
- 1 Cupful of Thick Sour Milk
- 1/2 Cupful of Sugar
- 1 Egg
- 2 Cupfuls of Flour
- 1/2 Cupful of Indian Meal
- 1 Teaspoonful of Baking Powder
- A pinch of Salt

**Instructions:**
1. Mix the ingredients until the mixture is stiff enough to drop from a spoon.
2. Drop the mixture, size of a walnut, into boiling fat.
3. Serve warm, with maple syrup.

---

### Boston Brown Bread
**Ingredients:**
- 1 Cupful of Rye Meal
- 1 Cupful of Graham Meal
- 1 Cupful of Indian Meal
- 1 Cupful of Flour
- 1 Cupful of Sour Milk
- 1 Cupful of Sweet Milk
- 1 Cupful of Molasses
- 1 Teaspoonful of Baking Soda

**Instructions:**
1. Mix all the ingredients together.
2. Beat the soda into the molasses until it foams; add the mixture to the rest.
3. Transfer into a tin pail which has been well greased.
4. If using a steamer, do not cover it tightly and steam for about 3 hours, but if not, bake in an oven until done.

Feel free to ask if you need more information or clarifications about these recipes!

In [ ]:
# Define improved system prompt
system_prompt2 = """
Please analyze the content of this image and extract any related recipe information into structure components.
Specifically, extra the recipe title, list of ingredients, step by step instructions, cuisine type, dish type, any relevant tags or metadata.
The output must be formatted in a way suited for embedding in a Retrieval Augmented Generation (RAG) system.
"""

In [ ]:
# Call the api to extract the information
response = client.chat.completions.create(
    model = model,
    messages = [
        # Provide the system prompt
        {"role": "system", "content": system_prompt2},

         # The user message contains both the text and image URL / path
        {"role": "user", "content": [
            {"type": "text", "text":"This is the image from the recipe page"},
            {"type": "image_url",
             "image_url": {"url": f"data:image/jpeg;base64,{image_data}",
                           "detail": "low"}}
        ]}
    ],
    temperature = 0, # Set the temperature to 0 for deterministic output
)

In [ ]:
# Print the info from the page with the improved prompt
get_gpt_response()

Based on the content of the image provided, here is the structured recipe information extracted:

```json
{
  "recipes": [
    {
      "title": "Bannocks",
      "ingredients": [
        "1 Cupful of Thick Sour Milk",
        "½ Cupful of Graham Meal",
        "½ Cupful of Flour",
        "½ Cupful of Indian Meal",
        "1 Teaspoonful of Salt",
        "1 Egg",
        "A pinch of Salt"
      ],
      "instructions": [
        "Make the mixture stiff enough to drop from a spoon.",
        "Drop mixture, size of a walnut, into boiling fat.",
        "Serve warm, with maple syrup."
      ],
      "cuisine_type": "American",
      "dish_type": "Bread",
      "tags": ["breads", "traditional", "easy"]
    },
    {
      "title": "Boston Brown Bread",
      "ingredients": [
        "1 Cupful of Rye Meal",
        "1 Cupful of Sour Milk",
        "1 Cupful of Graham Meal",
        "1 Cupful of Molasses",
        "½ Cupful of Flour",
        "1 Heaping Teaspoonful of Soda",
        "1 Cupful of Sweet Milk"
      ],
      "instructions": [
        "Mix the meal and salt together.",
        "Beat the soda into the molasses until it foams; add sour milk.",
        "Mix all together and pour into a tin pail which has been well greased, if you are not using a brown-bread steamer."
      ],
      "cuisine_type": "American",
      "dish_type": "Bread",
      "tags": ["breads", "traditional", "easy"]
    }
  ]
}
```

This structured format includes all relevant components for embedding in a Retrieval Augmented Generation (RAG) system, making it easy to retrieve and utilize the recipe information.

In [ ]:
# Extract information about all of the images/recipes
extracted_recipes = []

for image_path in image_paths:
  print(f"Processing image {image_path}")

  # Reading and decoding the image
  with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode("utf-8") # Encode the image to base64 format

  # Call the API to extract the information
  response = client.chat.completions.create(
      model = model,
      messages = [
          # Provide system prompt for guidance
          {"role": "system", "content": system_prompt2},

          # The user message contains both the text and image URL / path
          {"role": "user", "content": [
              {"type": "text", "text":"This is the image from the recipe page"}, # Context for the image
              {"type": "image_url",
              "image_url": {"url": f"data:image/jpeg;base64,{image_data}", # Provide the base64 image
                            "detail": "low"}}
          ]}
      ],
      temperature = 0, # Set the temperature to 0 for deterministic output
  )

  # Extract the content and store it
  gpt_response = response.choices[0].message.content # Get the response content
  extracted_recipes.append({"image_path": image_path, "recipe_info": gpt_response}) # Store the path and extracted info
  print(f"Extracted information for {image_path}:\n{gpt_response}\n") # Print the extracted information for review

Processing image images/page1.jpg
Extracted information for images/page1.jpg:
It seems that the image you provided is the cover of a book titled "Things Mother Used to Make" by Lydia Maria Gurney. Since this is a book cover and not a specific recipe, I cannot extract recipe information such as title, ingredients, instructions, or other related metadata.

If you have a specific recipe or page from the book that you would like me to analyze, please provide that information, and I would be happy to help!

Processing image images/page2.jpg
Extracted information for images/page2.jpg:
It seems that the image you provided does not contain any recipe information, such as a title, ingredients, or instructions. Instead, it appears to be a page indicating ownership of a book or document, specifically from the library of James B. Herndon, Jr., presented to the School of Hotel Administration at Cornell University.

If you have another image or more specific content related to a recipe, please share

RateLimitError: Error code: 429 - {'error': {'code': 'RateLimitReached', 'message': 'Rate limit of 150 per 86400s exceeded for UserByModelByDay. Please wait 18974 seconds before retrying.', 'details': 'Rate limit of 150 per 86400s exceeded for UserByModelByDay. Please wait 18974 seconds before retrying.'}}

In [ ]:
# Filter out non-recipe content based on key recipe-related terms
filtered_recipes = []

for recipe in extracted_recipes:
  # Check if the extracted content contains any key recipe-related terms
  if any(keyword in recipe["recipe_info"].lower() for keyword in ["ingredients",
                                                                  "instructions",
                                                                  "recipe title"]):
     # If it does, add it to the filtered list
    filtered_recipes.append(recipe)

  # Print a message for non-recipe content
  else:
    print(f"Skipping recipe: {recipe['image_path']}")

Skipping recipe: images/page3.jpg


In [ ]:
# import json library
import json

In [ ]:
# Define the output file path
output_file = "recipe_info.json"

# Write the filtered list to a json file
with open(output_file, "w") as json_file:
  json.dump(filtered_recipes, json_file, indent = 4)

# Embeddings

In [ ]:
# import libraries
import numpy as np

In [ ]:
# Load the filtered recipes
with open("recipe_info.json", "r") as json_file:
  filtered_recipes = json.load(json_file)

another options would be to organize per recipe, but it should be done in the preprocessing

In [ ]:
# Generate embeddings for each recipe
recipe_texts = [recipe["recipe_info"] for recipe in filtered_recipes] # Extract the text content of each recipe

# Call the API to generate embeddings for the recipe texts
embedding_response = client.embeddings.create(

    input = recipe_texts, # Provide the list of recipe texts as input
    model = "text-embedding-3-large" # Specify the embedding model to use
)

In [ ]:
# Extract the embeddings
embeddings = [data.embedding for data in embedding_response.data]
embeddings

[[-0.02960202842950821,
  -0.027912041172385216,
  -0.020715968683362007,
  -0.005114254541695118,
  -0.017813006415963173,
  -0.04770124331116676,
  5.321137359715067e-05,
  -0.015087221749126911,
  -0.010439757257699966,
  0.04061419889330864,
  -0.008184170350432396,
  -0.002255587372928858,
  -0.022664904594421387,
  0.00529824523255229,
  0.010950841940939426,
  -0.034181348979473114,
  -0.02291022427380085,
  0.012974738143384457,
  -0.028102846816182137,
  -0.04988187178969383,
  0.01054878905415535,
  0.009110936895012856,
  0.0484372042119503,
  -0.015932215377688408,
  0.03908776119351387,
  0.000801551213953644,
  -0.04159548133611679,
  -0.0015852145152166486,
  0.012272847816348076,
  0.0382155105471611,
  -0.004299926105886698,
  0.006081908475607634,
  0.03071960061788559,
  0.0034958196338266134,
  -0.018003812059760094,
  -0.04404868930578232,
  -0.01529165543615818,
  0.07463200390338898,
  0.005788886453956366,
  0.015359800308942795,
  0.01182990800589323,
  0.01459

In [ ]:
# Convert the embeddings to numpy array
embedding_matrix = np.array(embeddings)
embedding_matrix

array([[-2.96020284e-02, -2.79120412e-02, -2.07159687e-02, ...,
         9.90822911e-03, -2.44366657e-02,  9.69996254e-05],
       [-7.83261284e-03,  3.22135794e-03, -1.36530427e-02, ...,
         8.76418035e-03, -1.39064286e-02,  4.19205055e-03],
       [ 5.22245234e-03,  2.03441610e-04, -7.89824221e-03, ...,
         5.09691285e-03, -1.28767891e-02,  7.76194176e-03],
       [-6.60648476e-03, -1.30925942e-02, -8.03682767e-03, ...,
         3.17401602e-03, -1.43671576e-02,  7.94477575e-03],
       [ 9.38786659e-03, -1.03245405e-02, -1.21908449e-02, ...,
         5.89470658e-03, -1.33528840e-02,  1.48036722e-02],
       [-7.02451775e-03, -2.46418491e-02, -1.37164276e-02, ...,
        -2.36801803e-03, -2.75340844e-02,  1.93779804e-02]])

In [ ]:
# Verify the embedding matrix
print(f"Generated embeddings for {len(filtered_recipes)} recipes.")
print(f"Each embedding is of size {len(embeddings[0])}")

Generated embeddings for 6 recipes.
Each embedding is of size 3072


Each time we retrieve information, we may get different results

# Retrieval System

In [ ]:
# Install the faiss-cpu library
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 34.1 MB/s eta 0:00:00


In [ ]:
# Import the faiss library
import faiss

In [ ]:
# Print the embedding matrix shape
print(f"Embedding matrix shape: {embedding_matrix.shape}")

Embedding matrix shape: (6, 3072)


In [ ]:
# Initialize the FAISS index for similarity search
index = faiss.IndexFlatL2(embedding_matrix.shape[1]) # Create a FAISS index with L2 distance metric
index.add(embedding_matrix) # Add the embeddings to the index

In [ ]:
# Save the FAISS index to a file
faiss.write_index(index, "filtered_recipe_index.index")

In [ ]:
# Save the metadata for each recipe
metadata = [{'recipe_info': recipe['recipe_info'], # Include recipe information
             'image_path': recipe['image_path']} for recipe in filtered_recipes] # Include image path

# Write metadata to a JSON file with indentation
with open("recipe_metadata.json", "w") as json_file:
  json.dump(metadata, json_file, indent = 4)

In [ ]:
# Generate the embeddings for the query
query = "How to make bread?"
k = 5 # Number of top results to retrieve
query_embedding = client.embeddings.create(
    input = [query],
    model = "text-embedding-3-large"
).data[0].embedding
print(f"The query embedding is {query_embedding}\n")
query_vector = np.array(query_embedding).reshape(1, -1)  # Convert embedding to a 2D numpy array for FAISS
print(f"The query vector is {query_vector}\n")

# Search the FAISS index for the nearest neighbors
distances, indices = index.search(query_vector, min(k, len(metadata))) # Perform the search
print(f"The distances are {distances}\n")
print(f"The indices are {indices}\n")

# Store the indices and distances
stored_indices = indices[0].tolist()
stored_distances = distances[0].tolist()
print(f"The stored indices are {stored_indices}\n")
print(f"The stored distances are {stored_distances}\n")

# Print the metadata content for the top results
print("The metadata content is")
for i, dist in zip(stored_indices, stored_distances):
  if 0 <=i < len(metadata):
    print(f"Distance: {dist}, Metadata: {metadata[i]['recipe_info']}")

# Return the results
results = [(metadata[i]['recipe_info'], dist) for i, dist in zip(stored_indices, stored_distances) if 0 <= i < len(metadata)]
results # Output the results as a list of tuples containing recipe info and distance

The query embedding is [-0.019720381125807762, -0.028134725987911224, -0.022066915407776833, 0.016603518277406693, -0.04792621359229088, -0.04887430742383003, 0.038018617779016495, 0.017954552546143532, 0.0015480617294088006, 0.004168656188994646, -0.004832322709262371, -0.013889594934880733, -0.012917797081172466, -0.0015850967029109597, 0.038018617779016495, -0.0238090418279171, 0.017018308863043785, -0.016674624755978584, -0.0072233001701533794, 0.002310981974005699, -0.031856000423431396, 0.02015887387096882, 0.01501545775681734, -0.007821785286068916, 0.006275205407291651, 0.01784789189696312, 0.003410179866477847, -0.011229002848267555, -0.03960667923092842, 0.042285047471523285, -0.0036649806424975395, 0.023678677156567574, -0.034676581621170044, -0.02115437388420105, -0.015904298052191734, 0.011306035332381725, -0.01473102904856205, 0.005285630933940411, 0.00849730335175991, 0.019412249326705933, -0.01242004707455635, 0.007833636365830898, 0.00040108870598487556, -0.00825435388

[('It seems that the image you provided is the cover of a book titled "Things Mother Used to Make" by Lydia Maria Gurney. Since this is a book cover and not a specific recipe, I cannot extract recipe information such as title, ingredients, instructions, or other related metadata.\n\nIf you have a specific recipe or page from the book that you would like me to analyze, please provide that information, and I would be happy to help!',
  1.5542783737182617),
 ('It seems that the image you provided does not contain any specific recipe information, such as a title, ingredients, instructions, or any other relevant details. The text visible in the image is simply "THINGS MOTHER USED TO MAKE," which does not provide enough context to extract structured recipe components.\n\nIf you have another image or more detailed information about a specific recipe, please share it, and I would be happy to help you extract the relevant details!',
  1.5582467317581177),
 ('It seems that the image you provided

In [ ]:
# Define a function to query the embeddings
def query_embeddings(query, index, metadata, k = 5):
  # Generate the embeddings for the query
  query_embedding = client.embeddings.create(
      input = [query],
      model = "text-embedding-3-large"
  ).data[0].embedding
  print(f"The query embedding is {query_embedding}\n")
  query_vector = np.array(query_embedding).reshape(1, -1)
  print(f"The query vector is {query_vector}\n")

  # Search faiss index
  distances, indices = index.search(query_vector, min(k, len(metadata)))
  # print(f"The distances are {distances}\n")
  # print(f"The indices are {indices}\n")

  # Store the indices and distances
  stored_indices = indices[0].tolist()
  stored_distances = distances[0].tolist()
  print(f"The stored indices are {stored_indices}\n")
  print(f"The stored distances are {stored_distances}\n")

  # # Print the metadata content
  # print("The metadata content is")
  # for i, dist in zip(stored_indices, stored_distances):
  #   if 0 <=i < len(metadata):
  #     print(f"Distance: {dist}, Metadata: {metadata[i]['recipe_info']}")

  # Return the results
  results = [(
      metadata[i]['recipe_info'], dist) for i, dist in zip(
          stored_indices, stored_distances) if 0 <= i < len(metadata)]
  return results


In [ ]:
# Test the retrieval system
query = "chocolate query"
results = query_embeddings(query, index, metadata)
print(f"The results are {results}")

The query embedding is [-0.014626344665884972, -0.005924540106207132, -0.01682029664516449, -0.006895276717841625, -0.0328744500875473, 0.009994668886065483, 0.023471800610423088, 0.0519583486020565, -0.015261894091963768, 0.046490881592035294, 0.005841831676661968, -0.01802174560725689, -0.049799222499132156, 0.0076222410425543785, -0.025787638500332832, 0.014669875614345074, -0.013781847432255745, -0.015174832195043564, -0.020424645394086838, -0.03021036647260189, 0.02106890082359314, -0.01762126199901104, -0.005419583059847355, 0.02754628285765648, 0.023367326706647873, 0.04920720309019089, -0.011326710693538189, -0.00455767335370183, -0.038620512932538986, 0.012397568672895432, 0.01265004649758339, 0.018718238919973373, 0.027911940589547157, -0.021591270342469215, -0.02025052160024643, 0.009750896133482456, 0.01760384999215603, -0.038829460740089417, 0.015897443518042564, -0.009315588511526585, 0.02211363986134529, -0.03590419515967369, -0.021643508225679398, -0.0011677134316414595

In [ ]:
# Combine the results into a single string to feed to gpt
def combined_retrived_content(results):
  combined_content = "\n\n".join([result[0] for result in results]) # Join the recipe information with double newlines
  return combined_content

# Get the combined content from results
combined_content = combined_retrived_content(results)
print(f"The combined content is {combined_content}")

The combined content is It seems that the image you provided is the cover of a book titled "Things Mother Used to Make" by Lydia Maria Gurney. Since this is a book cover and not a specific recipe, I cannot extract recipe information such as title, ingredients, instructions, or other related metadata.

If you have a specific recipe or page from the book that you would like me to analyze, please provide that information, and I would be happy to help!

Based on the content of the image provided, here is the structured information extracted for embedding in a Retrieval Augmented Generation (RAG) system:

```json
{
  "recipe_title": "Things Mother Used to Make",
  "author": "Lydia Maria Gurney",
  "publication_year": 1922,
  "cuisine_type": "Traditional",
  "dish_type": "Collection of Recipes",
  "ingredients": [],
  "instructions": [],
  "tags": [
    "old-time recipes",
    "traditional cooking",
    "historical recipes",
    "family recipes"
  ],
  "metadata": {
    "publisher": "The Mac

# Generative System

In [ ]:
# Define the system prompt
system_prompt3 = f"""
You are highly experienced and expert chef specialized in providing cooking advice.
Your main task is to provide information precise and accurate on the combined content.
You answer diretly to the query using only information from the provided {combined_content}.
If you don't know the answer, just say that you don't know.
Your goal is to help the user and answer the {query}
"""

In [ ]:
# Define function to retrieve a response from the API
def generate_response(query, combined_content, system_prompt):
  response = client.chat.completions.create(
      model = model,
      messages = [
          {"role": "system", "content": system_prompt3}, # Provide system prompt for guidance
          {"role": "user", "content": query}, # Provide the query as user input
          {"role": "assistant", "content": combined_content} # Provide the combined content from the results
      ],
      temperature = 0, # Set temperature to 0 for deterministic output
  )
  return response

In [ ]:
# Get the results from the API
query = "How to make bread?"
combined_content = combined_retrived_content(results)
response = generate_response(query, combined_content, system_prompt3)

RateLimitError: Error code: 429 - {'error': {'code': 'RateLimitReached', 'message': 'Rate limit of 150 per 86400s exceeded for UserByModelByDay. Please wait 10456 seconds before retrying.', 'details': 'Rate limit of 150 per 86400s exceeded for UserByModelByDay. Please wait 10456 seconds before retrying.'}}

In [ ]:
# Display the outcome
get_gpt_response()

I'm sorry, but the provided content does not include a recipe for making bread. If you have a specific bread recipe in mind or need guidance on a particular type of bread, please let me know!

In [ ]:
# Get the results
query = "Get me the best chocolate cake recipe"
combined_content = combined_retrived_content(results)
response = generate_response(query, combined_content, system_prompt3)

In [ ]:
# Display the outcome
get_gpt_response()

I'm sorry, but I don't have a specific chocolate cake recipe available. However, I can provide you with a chocolate sauce recipe if you're interested in making a sauce to accompany a cake. Would you like that?

# Rag system

In [ ]:
# Build the function for Retrieval-Augmented Generation (RAG)
def rag_system(query, index, metadata, system_prompt, k = 5):
  # Retrieval System: Retrieve relevant results based on the query
  results = query_embeddings(query, index, metadata, k)

  # Content Merge: Combine the retrieved content into a single string
  combined_content = combined_retrived_content(results)

  # Generation: Generate a response based on the query and combined content
  response = generate_response(query, combined_content, system_prompt)

  # Return the generated response
  return response

In [ ]:
# Test the rag system
query1 = "How to make the best chocolate cake?"
response = rag_system(query1, index, metadata, system_prompt3)
get_gpt_response()

The query embedding is [-0.0025930912233889103, -0.03965035080909729, -0.013300711289048195, 0.0075972783379256725, -0.019310662522912025, 0.022375846281647682, 0.00811179168522358, -0.005364072974771261, -0.006425940431654453, 0.03741714358329773, -0.0004070948052685708, 0.007121080067008734, -0.051801618188619614, -0.007602752186357975, 0.0003212285810150206, -0.0403071753680706, 0.02113882638514042, -0.004690826870501041, -0.004521146882325411, -0.015851382166147232, 0.014964668080210686, -0.024806099012494087, -0.011735277250409126, 0.03402354568243027, 0.0030405535362660885, 0.006721511483192444, 0.022551000118255615, -0.0006903507164679468, -0.03689168393611908, 0.056793488562107086, 0.04457654058933258, 0.0018993195844814181, -0.01622358337044716, -0.014745726250112057, -0.029163040220737457, 0.01690230332314968, -0.004753772635012865, -0.0019444763893261552, 0.03478984162211418, 0.01440636720508337, -0.005741747096180916, 4.0773622458800673e-05, 0.0025917228776961565, 0.0107719

To make the best chocolate cake, you can follow this classic recipe:

### Recipe Title: Chocolate Cake

### Ingredients:
- 1 ¾ cups all-purpose flour
- 1 ¾ cups granulated sugar
- ¾ cup unsweetened cocoa powder
- 1 ½ teaspoons baking powder
- 1 ½ teaspoons baking soda
- 1 teaspoon salt
- 2 large eggs
- 1 cup whole milk
- ½ cup vegetable oil
- 2 teaspoons vanilla extract
- 1 cup boiling water

### Instructions:
1. **Preheat the Oven**: Preheat your oven to 350°F (175°C). Grease and flour two 9-inch round cake pans.
   
2. **Mix Dry Ingredients**: In a large mixing bowl, combine the flour, sugar, cocoa powder, baking powder, baking soda, and salt. Whisk together until well combined.

3. **Add Wet Ingredients**: Add the eggs, milk, vegetable oil, and vanilla extract to the dry ingredients. Beat on medium speed for about 2 minutes until well blended.

4. **Incorporate Boiling Water**: Carefully stir in the boiling water (the batter will be thin). Mix until smooth.

5. **Bake**: Pour the batter evenly into the prepared cake pans. Bake for 30 to 35 minutes, or until a toothpick inserted in the center comes out clean.

6. **Cool**: Allow the cakes to cool in the pans for 10 minutes, then remove from pans and transfer to wire racks to cool completely.

7. **Frosting**: Once cooled, frost with your favorite chocolate frosting.

### Cuisine Type:
American

### Dish Type:
Dessert

### Relevant Tags/Metadata:
- Chocolate Cake
- Baking
- Dessert
- Sweet Treat

Enjoy your delicious homemade chocolate cake! If you need any variations or additional tips, feel free to ask!

In [ ]:
# Test with a different query
query2 = "I want something vegan"
response = rag_system(query2, index, metadata, system_prompt3)
get_gpt_response()

The query embedding is [-0.034529902040958405, -0.026633311063051224, -0.017109284177422523, 0.03441669046878815, -0.01732155680656433, -0.008498035371303558, 3.413520607864484e-05, -0.01890653744339943, -0.033737413585186005, 0.02354826219379902, 0.003265481675043702, -0.012984092347323895, 0.0179583802819252, -0.0071642473340034485, 0.036850765347480774, -0.008830597624182701, 0.0097221489995718, -0.011399113573133945, 0.007479120511561632, -0.0183546245098114, -0.023519957438111305, 0.006028581410646439, 0.0375017374753952, -0.006902442779392004, 0.007719697430729866, 0.00817254837602377, -0.022090647369623184, -0.012241133488714695, 0.002090898808091879, 0.018467837944626808, 0.010189151391386986, 0.024496419355273247, -0.005041507072746754, 0.055304449051618576, -0.013684596866369247, -0.003852772992104292, 0.03874707967042923, -0.0019228485180065036, 0.01562336552888155, 0.026958797127008438, -0.047747496515512466, 0.022005736827850342, -0.02418508380651474, 0.001378896413370967,

It seems that the recipes provided do not include any vegan options. However, if you're looking for vegan alternatives or suggestions, I can help you create a vegan recipe or modify an existing one. Please let me know what type of dish you're interested in, and I'll be happy to assist!